In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null


In [6]:
%%writefile ATM.java

import java.time.LocalDateTime;
import java.time.format.DateTimeFormatter;
import java.util.ArrayList;
import java.util.HashMap;
import java.util.List;
import java.util.Map;
import java.util.Scanner;


public class ATM {
    private static final BankDatabase bankDatabase = new BankDatabase();
    private static final ATMOperations operations = new ATMOperations();
    private static final Scanner scanner = new Scanner(System.in);

    public static void main(String[] args) {
        System.out.println("Welcome to the ATM! 🏧");

        // The main loop keeps the program running until explicitly quit by a user.
        while (true) {
            System.out.println("\nPlease insert your card (Enter your User ID):");
            String userId = scanner.nextLine();

            System.out.println("Enter your PIN:");
            String pin = scanner.nextLine();

            // Authenticate the user
            UserAccount currentUser = bankDatabase.authenticateUser(userId, pin);

            if (currentUser != null) {
                System.out.println("\nHello, " + userId + "! Login successful.");
                showMenu(currentUser);
            } else {
                System.out.println("❌ Invalid User ID or PIN. Please try again.");
            }
        }
    }


    private static void showMenu(UserAccount account) {
        boolean userIsActive = true;
        while (userIsActive) {
            System.out.println("\nWhat would you like to do?");
            System.out.println("  1. Transactions History");
            System.out.println("  2. Withdraw");
            System.out.println("  3. Deposit");
            System.out.println("  4. Transfer");
            System.out.println("  5. Quit");
            System.out.print("Enter your choice: ");

            String choice = scanner.nextLine();

            switch (choice) {
                case "1":
                    operations.showTransactionHistory(account);
                    break;
                case "2":
                    operations.withdraw(account, scanner);
                    break;
                case "3":
                    operations.deposit(account, scanner);
                    break;
                case "4":
                    operations.transfer(account, bankDatabase, scanner);
                    break;
                case "5":
                    userIsActive = false; // Exit the menu loop
                    System.out.println("\nThank you for using the ATM. Goodbye! 👋");
                    break;
                default:
                    System.out.println("Invalid choice. Please select an option from 1-5.");
            }
        }
    }
}

// ===== SUPPORTING CLASSES =====


class ATMOperations {

    public void showTransactionHistory(UserAccount account) {
        System.out.println("\n--- Transaction History for " + account.getUserId() + " ---");
        List<Transaction> history = account.getTransactionHistory();
        if (history.isEmpty()) {
            System.out.println("No transactions found.");
        } else {
            for (Transaction t : history) {
                System.out.println(t);
            }
        }
    }

    public void withdraw(UserAccount account, Scanner scanner) {
        System.out.print("Enter amount to withdraw: ₹");
        try {
            double amount = Double.parseDouble(scanner.nextLine());
            if (amount <= 0) {
                System.out.println("Withdrawal amount must be positive.");
                return;
            }
            if (account.getBalance() >= amount) {
                account.withdraw(amount);
                System.out.println("Withdrawal successful. Current balance: ₹" + account.getBalance());
            } else {
                System.out.println("Insufficient funds. Your balance is ₹" + account.getBalance());
            }
        } catch (NumberFormatException e) {
            System.out.println("Invalid amount. Please enter a number.");
        }
    }

    public void deposit(UserAccount account, Scanner scanner) {
        System.out.print("Enter amount to deposit: ₹");
        try {
            double amount = Double.parseDouble(scanner.nextLine());
            if (amount <= 0) {
                System.out.println("Deposit amount must be positive.");
                return;
            }
            account.deposit(amount);
            System.out.println("Deposit successful. Current balance: ₹" + account.getBalance());
        } catch (NumberFormatException e) {
            System.out.println("Invalid amount. Please enter a number.");
        }
    }

    public void transfer(UserAccount fromAccount, BankDatabase db, Scanner scanner) {
        System.out.print("Enter recipient's User ID: ");
        String toUserId = scanner.nextLine();
        UserAccount toAccount = db.getAccount(toUserId);

        if (toAccount == null) {
            System.out.println("Recipient account not found.");
            return;
        }

        if(fromAccount.getUserId().equals(toUserId)){
            System.out.println("You cannot transfer money to your own account.");
            return;
        }

        System.out.print("Enter amount to transfer: ₹");
        try {
            double amount = Double.parseDouble(scanner.nextLine());
            if (amount <= 0) {
                System.out.println("Transfer amount must be positive.");
                return;
            }
            if (fromAccount.getBalance() >= amount) {
                fromAccount.transfer(toAccount, amount);
                System.out.println("Transfer successful. Your new balance: ₹" + fromAccount.getBalance());
            } else {
                System.out.println("Insufficient funds for transfer. Your balance is ₹" + fromAccount.getBalance());
            }
        } catch (NumberFormatException e) {
            System.out.println("Invalid amount. Please enter a number.");
        }
    }
}


class BankDatabase {
    private final Map<String, UserAccount> accounts;

    public BankDatabase() {
        accounts = new HashMap<>();
        // Add some dummy accounts to our "database"
        accounts.put("user1", new UserAccount("user1", "1234", 50000.0));
        accounts.put("user2", new UserAccount("user2", "5678", 25000.0));
    }

    public UserAccount authenticateUser(String userId, String pin) {
        UserAccount account = accounts.get(userId);
        if (account != null && account.validatePin(pin)) {
            return account;
        }
        return null;
    }

    public UserAccount getAccount(String userId) {
        return accounts.get(userId);
    }
}


class UserAccount {
    private final String userId;
    private final String pin;
    private double balance;
    private final List<Transaction> transactionHistory;

    public UserAccount(String userId, String pin, double balance) {
        this.userId = userId;
        this.pin = pin;
        this.balance = balance;
        this.transactionHistory = new ArrayList<>();
    }

    // Simple getters
    public String getUserId() { return userId; }
    public double getBalance() { return balance; }
    public List<Transaction> getTransactionHistory() { return transactionHistory; }

    public boolean validatePin(String inputPin) {
        return this.pin.equals(inputPin);
    }

    public void deposit(double amount) {
        this.balance += amount;
        addTransaction("Deposit", amount);
    }

    public void withdraw(double amount) {
        this.balance -= amount;
        addTransaction("Withdrawal", amount);
    }

    public void transfer(UserAccount toAccount, double amount) {
        // Withdraw from this account
        this.balance -= amount;
        addTransaction("Transfer to " + toAccount.getUserId(), amount);

        // Deposit into the other account
        toAccount.balance += amount;
        toAccount.addTransaction("Transfer from " + this.userId, amount);
    }

    private void addTransaction(String type, double amount) {
        this.transactionHistory.add(new Transaction(type, amount));
    }
}



class Transaction {
    private final String type;
    private final double amount;
    private final String timestamp;

    public Transaction(String type, double amount) {
        this.type = type;
        this.amount = amount;
        // Let's create a nicely formatted timestamp
        this.timestamp = LocalDateTime.now().format(DateTimeFormatter.ofPattern("dd-MM-yyyy HH:mm:ss"));
    }

    @Override
    public String toString() {
        return String.format("[%s] %s: ₹%.2f", timestamp, type, amount);
    }
}


Writing ATM.java


In [7]:
!javac ATM.java


In [ ]:
!java ATM


Welcome to the ATM! 🏧

Please insert your card (Enter your User ID):
user1
Enter your PIN:
1234

Hello, user1! Login successful.

What would you like to do?
  1. Transactions History
  2. Withdraw
  3. Deposit
  4. Transfer
  5. Quit
Enter your choice: 2
Enter amount to withdraw: ₹4000
Withdrawal successful. Current balance: ₹46000.0

What would you like to do?
  1. Transactions History
  2. Withdraw
  3. Deposit
  4. Transfer
  5. Quit
Enter your choice: 1

--- Transaction History for user1 ---
[01-09-2025 09:11:43] Withdrawal: ₹4000.00

What would you like to do?
  1. Transactions History
  2. Withdraw
  3. Deposit
  4. Transfer
  5. Quit
Enter your choice: 3
Enter amount to deposit: ₹5000
Deposit successful. Current balance: ₹51000.0

What would you like to do?
  1. Transactions History
  2. Withdraw
  3. Deposit
  4. Transfer
  5. Quit
Enter your choice: 5

Thank you for using the ATM. Goodbye! 👋

Please insert your card (Enter your User ID):
exit
Enter your PIN:
1234
❌ Invalid Use